In [ ]:
import pickle
from docuverse.utils import open_stream
from pymilvus import (
    MilvusClient,
    DataType,
    connections,
    utility
)
import numpy as np

In [ ]:
MODEL = ("/home/raduf/sandbox2/docuverse/models/slate.30m.english.rtrvr")
data_list = []
from docuverse.utils.embeddings.dense_embedding_function import DenseEmbeddingFunction
model = DenseEmbeddingFunction(MODEL)

In [ ]:
client=MilvusClient("http://localhost:19530")

In [ ]:
client.list_collections()

In [97]:
ddata = [
    {"text": "Apply these configuration details to nodes to ensure that you have a valid configuration. Host bus adapters and node canisters Each of the FlashSystem 7200 node canisters feature the optional adapters that are shown in this table. Table 1. Host interface adapters Supported number of adapters Ports Protocol Possible slots 0-2 4 32 Gb Fibre Channel 1,2 0-2 4 16 Gb Fibre Channel 1,2 0-2 2 25 Gb Ethernet (iWARP) 1,2 0-2 2 25 Gb Ethernet (RoCE) 1,2 0-1 in slot 1 only 2 (4-port adapter but only 2 ports are active.) 12 Gb SAS Expansion 1 FlashSystem 7200, IBM\u00ae Storage FlashSystem 9100, and FlashSystem 9200 can be connected over 16 or 32 Gbps Fibre Channel or 25 Gbps Ethernet. The 32 Gbps Fibre Channel adapter supports simultaneous SCSI and NVMeFC connections on the same port. The SAS expansion adapter is required to use 2076-24G or 4664-24G or 2076-92G or 4664-92G expansion enclosures. The 25 Gb adapters support iSCSI host attachment. FlashSystem 7200 node canisters also contain two USB ports and the on-board Ethernet ports shown in this table. Table 2. On-board Ethernet ports On-board Ethernet port Speed Functions 1 10 GbE Management IP, Service IP, Host I/O 2 10 GbE Secondary Management IP, Host I/O 3 10 GbE Host I/O 4 10 GbE Host I/O 1 GbE Technician Port - DHCP/DNS for direct attach service management Fibre Channel over Ethernet (FCoE) is not supported. For information about supported hardware, see the following website: www.ibm.com/support . Volumes Each volume is accessible from the two nodes in an I/O group. Each host network port can recognize up to eight paths to each logical unit (LU) that is presented by the system. The hosts must run a multipathing device driver before the multiple paths can resolve to a single device. ", "id": "288534-1-1755"},
    {"text": "HyperSwap configuration by using interswitch links\nNote: ISLs must not be shared between private and public virtual fabrics. To implement private and public SANs with dedicated switches, any combination of supported switches can be used. For the list of supported switches and for supported switch partitioning and virtual fabric options, see the interoperability website: www.ibm.com/support Like for every managed disk, all control enclosures need access to the quorum disk by using the same storage system ports. If a storage system with active/passive controllers (such as IBM DS3000, IBM DS4000\u00ae, IBM DS5000, or IBM FAStT) is attached to a fabric, the storage system must be connected with both internal controllers to this fabric. By using FCIP, passive WDM, or active WDM for quorum site connectivity, you can add to the extension. The connections must be reliable. It is strictly required that the links from both production sites to the quorum site are independent and do not share any long-distance equipment. FCIP links are supported also for ISLs between the two production sites in public and private SANs. A private SAN and a public SAN can be routed across the same FCIP link. However, to ensure bandwidth to the private SAN, it is typically necessary to configure FCIP tunnels. Similarly, it is permissible to multiplex multiple ISL links across a DWDM link. Note: It is not required to UPS-protect FCIP routers or active WDM devices that are used only for the control enclosure-to-quorum communication. A HyperSwap configuration is supported only when the storage system that hosts the quorum disks supports extended quorum. Although the system can use other types of storage systems for providing quorum disks, access to these quorum disks is always through a single path. Additional bandwidth requirements A bandwidth equal to the peak write bandwidth (as sum from all hosts) is required for intersite communication between I/O groups. This bandwidth must be available in the private SAN. Additionally, you need intersite bandwidth in the public SAN for host-to-node communication if a host accesses nodes in the other sites. ", "id": "365530-3382-5476"},
    {"text": "is it possible to do a hyperswap configuration between fs7300 and fs5200", "id": "test1",},
    {"text": "does current virtualize version support remote mirror / replication encryption", "id": "q2"}
]

def create_data(raw_data, embedding_name='qembedding'):
    embeddings = model.encode([d['text'] for d in raw_data], show_progress_bar=True)
    keys_to_keep = {"text"}
    data_list = [{**{k:v for k, v in d.items() if k in keys_to_keep}, '_id': d['id'], embedding_name:embeddings[i]} for i,d in enumerate(raw_data)]
    return data_list
data_list = create_data(ddata)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def cosine(l1, l2, name="qembedding"):
    return np.inner(l1[name], l2[name])

cosine(data_list[0], data_list[2])

In [ ]:
cosine(data_list[1], data_list[2])

In [40]:
[c for c in client.list_collections() if c.find("dev") >= 0]

['ibmsw_dev_short_milvus_dense_slate30m_512_100_10062024',
 'ibmsw_dev_short_ndcg_milvus_dense_slate30m_512_100_10062024',
 'ibmsw_dev_short_milvus_dense_slate30m_512_100_10252024']

In [101]:
def search(index_name, vector, top=1):
    return client.search(collection_name=index_name, data=[vector['qembedding']], limit=top, 
                         params={"metric_type": "IP", "params": {"ef": 100}}, output_fields=[ 'id'])[0]

index     ='ibmsw_dev_short_milvus_dense_slate30m_512_100_10252024'
ndcg_index='ibmsw_dev_short_ndcg_milvus_dense_slate30m_512_100_10252024'

s1=search(index, data_list[3], top=3)
s2=search(ndcg_index, data_list[3], top=3)

In [106]:
[s['entity'] for s in s1] == [s['entity'] for s in s2]

True

In [103]:
s2

[{'id': 453475059162090949,
  'distance': 0.728019118309021,
  'entity': {'id': '360459-0-217'}},
 {'id': 453475059162127226,
  'distance': 0.7124879360198975,
  'entity': {'id': '1173769-2169-3720'}},
 {'id': 453475059162124325,
  'distance': 0.711053729057312,
  'entity': {'id': '1401329-1695-2946'}}]

In [80]:
client.get_collection_stats(index)

{'row_count': 88945}